# [LEGALST-123] Lab 05: Large N and Hypothesis Testing

This lab will cover the basics of statistical sampling, the law of averages, and hypothesis testing. You should gain an intuition around how samples relate to populations, and the basics of statistical inference in the social sciences.

In [ ]:
from collections import Counter
import numpy as np
import pandas as pd
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plot
plot.style.use('fivethirtyeight')

## Data

We'll continue using the ANES data for this lab!

In [ ]:
anes = pd.read_csv('../data/anes/ANES_legalst123_cleaned.csv')
anes.head()

## Empirical Distributions

### Data Manipulation and Plotting Review

Let's look at how liberal respondents characterized themselves as post-election. Write code that saves the "post_liberal_rating" column in the ANES data to a Series variable. Keep in mind that valid answers have domain [0,100] so be sure to subset to only those values.

Plot a histogram of the data:

### Question 1

What patterns do you notice? Where is the center of the distribution? What does this suggest about how Americans tend to self-identify?

Answer:

### Law of Averages

Write a function, "empirical_hist_anes" that takes a Series and a sample size as its argument, and then draws a histogram based on the results. Consult Adhikari and DeNero for help!

In [ ]:
def empirical_hist_anes(series, n):
    ...

Check how many rows are in the table with the "size" method, and then use your self-defined function to plot histograms taking sample sizes 10, 100, 1000, and the total number of rows.

### Question 2

What happens to the histograms (compared to the original in Q1) as you increase the sample size? How does this relate to the Law of Averages? What is the relationship between sample size and population parameter estimation?

## Hypothesis Testing

In this section, we'll cover the basic tools for hypothesis testing. 

The goal in conducting a hypothesis test is to answer the question, "Was it likely to observe my test statistic due to chance?" We say something is statistically significant if it is sufficiently far enough away from the center of an empirical distribution, and therefore unlikely to have occurred just by chance.

The basic way to frame a hypothesis test is as follows:

1. Define a null $(H_O)$ and alternative $(H_A)$ hypothesis. The null hypothesis is usually framed as "no statistical relationship between the observed data and the background distribution" and the alternative hypothesis is the opposite. More concretely, the null is our default position, and assumes that the observed statistic likely came from the background distribution.

2. Calculate a test statistic (for example, t-test, $\chi^2$, etc.)

3. Check if the test statistic is far enough away from the center of the distribution. Traditionally, this was done by checking against a reference table, but in Python, we'll use p-values. Typically, a p value of less than .05 (meaning that only 5% of observations should fall where the test statistic does) is used as the threshold for statistical significance in the social sciences.

4. Either reject or fail to reject the null hypothesis.

### Jury Selection

First, we'll use the jury selection example from the Adhikari and DeNero book. This example is based on the U.S. Supreme Court case, Swain v. Alabama. Robert Swain was convicted by an all-white jury, and challenged his conviction on the basis that it was statistically unlikely that a jury would be all-white by chance, given that the racial composition of the county was 18% black. Juries were selected from a panel of 100. In this case, only 8 jurors on the panel were black.

Was it likely that the panel would only include 8 black jurors out of 100, given that 18% of the county was black?

In [ ]:
# Create the table
jury = pd.DataFrame(data = {'Ethnicity': ['Asian', 'Black', 'Latino', 'White', 'Other'],
                           'Eligible': [0.15, 0.18, 0.12, 0.54, 0.01],
                           'Panels': [0.26, 0.08, 0.08, 0.54, 0.04]}
)

jury

In [ ]:
# Horizontal Bar Chart
jury.plot.barh('Ethnicity')

In [ ]:
# Augment with the difference between the "panels" columns and "eligible" column
jury_with_diffs = jury.assign(Difference = jury.loc[:, 'Panels'] - jury.loc[:, 'Eligible'])
jury_with_diffs

**Testing the hypothesis:**
In this scenario, the null hypothesis is that the jury panel was drawn uniformly from the eligible jury pool. Think of this as a statistical model. We will ask: under a model of uniform jury selection, how likely is it that Swain's jury panel would be selected? If we find it to be sufficiently unlikely, you may conclude that the model does not describe reality, i.e. that his jury panel was not selected uniformly.

**Simulate selection of 1000 jury panels uniformly from the eligible jury pool.** This should generate a DataFrame with a column for each race, and each row describing the proportion of each race represented in one jury panel.

In [1]:
def sample_proportions(probabilities, samples):
    # You may have used datascience.sample_proportions in Data 8; this is how it works.
    # Read more here https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.multinomial.html
    return np.random.multinomial(samples, probabilities) / samples



**Visualize the distribution for each race.** As a first look, how common would a panel like the one in Swain v. Alabama appear to be?

We can't quite compute a t-test yet. We have a set of samples from our simulation of colorblind jury selection, and we have a single sample of a real jury panel. t-tests are for comparing a pair of sample sets which each have a large number of samples.

Ideally, we would have use the set of real jury panels from the jurisdiction in question as another sample set, and run a t-test between that sample set and the 1000 simulated jury panels. As a *substitute* for this, assume that the particular jury panel in Swain's case is *representative* of jury panels in the district (ask yourself: is this a fair assumption?); simulate the selection of jury panels with a mean of `jury['Panel']`, and run a t-test between those juries and your simulated fair juries. You should get very confident p-values. Why?

Hint: use https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind

Since the legal question in this case is concerned with specifically the representation of black jurors, specifically on Swain's jury panel, we can use some simpler statistics. **Visualize the distribution of the 'Black' column in the simulated jury panels, and compute the portion of simulated jury panels having under 9% black jurors.**

### Hypothesis Testing on ANES Data

Now let's try with the ANES data! Write code that creates a new DataFrame with the "post_liberal_rating" and "post_conservative_rating" as columns, and only includes values below 150.

### Question 3

Plot a histogram of both the post liberal rating and post conservating rating side by side. Experiment with different bin widths. Visually, what can you infer about the shape of each data?

### Question 4

Now write code to do a t-test for Democrat vs Republican vs Independent feeling thermometer scores toward liberals, as well as a t-test for Democrat vs Republican vs Independent feeling thermometer scores toward conservative.

For the t-test to work, you have to remove NaN values first.

In [ ]:
# Drop NaN

Note that `pre_party_id` (`V161158x` in codebook) is using the 7-pt scale summary measure. We need to recode `pre_party_id` so that it is just Democrat, Republican, Independent.

In [ ]:
parties = {
    1.0: "dem",
    2.0: "dem",
    3.0: "dem",
    4.0: "ind",
    5.0: "rep",
    6.0: 'rep',
    7.0: 'rep'
}
liberal_v_conservative["pre_party_id"] = liberal_v_conservative["pre_party_id"].apply(lambda ...)
liberal_v_conservative['pre_party_id'].hist()

Now we can do a boxplot to visualize the distributions of liberal feeling thermometer scores and conservative feeling thermometer scores for each party.

In [ ]:
liberal_v_conservative.boxplot(column='post_liberal_rating', by=..., figsize=(9,6))

liberal_v_conservative.boxplot(column='post_conservative_rating', by=..., figsize=(9,6))

Now we can do a two sample t-test to see whether we can reject the null hypothesis at a 95% confidence level that partisan identifiers represent the same underlying population.

In [ ]:
# t-test for Democrat vs Republican vs Independent feeling thermometer scores toward liberals

# first, for clarity, get the series of feeling thermometer ratings on liberals for dems, reps, inds
dems_on_liberal = liberal_v_conservative[liberal_v_conservative['pre_party_id']=='dem']['post_liberal_rating']
reps_on_liberal = liberal_v_conservative[liberal_v_conservative['pre_party_id']=='rep']['post_liberal_rating']
inds_on_liberal = liberal_v_conservative[liberal_v_conservative['pre_party_id']=='ind']['post_liberal_rating']

print("mean Democrat feeling thermometer toward liberals", dems_on_liberal.mean())
print("mean Republican feeling thermometer toward liberals", reps_on_liberal.mean())
print("mean Independent feeling thermometer toward liberals", inds_on_liberal.mean())

print("two sample t-test for Dems & Reps ", stats.ttest_ind(..., ..., equal_var = False, nan_policy='raise'))
print("two sample t-test for Dems & Inds ", stats.ttest_ind(..., ..., equal_var = False, nan_policy='raise'))
print("two sample t-test for Inds & Reps ", stats.ttest_ind(..., ..., equal_var = False, nan_policy='raise'))

In [ ]:
# now let's do t-test for Democrat vs Republican vs Independent feeling thermometer scores toward conservative
dems_on_conserv = liberal_v_conservative[liberal_v_conservative['pre_party_id']=='dem']['post_conservative_rating']
reps_on_conserv = liberal_v_conservative[liberal_v_conservative['pre_party_id']=='rep']['post_conservative_rating']
inds_on_conserv = liberal_v_conservative[liberal_v_conservative['pre_party_id']=='ind']['post_conservative_rating']

print("mean Democrat feeling thermometer toward conservatives", dems_on_conserv.mean())
print("mean Republican feeling thermometer toward conservatives", reps_on_conserv.mean())
print("mean Independent feeling thermometer toward conservatives", inds_on_conserv.mean())

print("two sample t-test for Dems & Reps ", stats.ttest_ind(..., ..., equal_var = False, nan_policy='raise'))
print("two sample t-test for Dems & Inds ", stats.ttest_ind(..., ..., equal_var = False, nan_policy='raise'))
print("two sample t-test for Inds & Reps ", stats.ttest_ind(..., ..., equal_var = False, nan_policy='raise'))

## Central Limit Theorem

The central limit theorem (CLT) is a fundamental concept in statistics. It basically says that the means of repeated samples will converge upon a normal distribution centered around the population mean. This is a powerful result that allows us to use a sample mean without measuring other sample means. This insight is particularly important in the social sciences, and justifies the use of regression for causal inference.

Using liberal respondents ("post_liberal_rating") again, let's illustrate this concept. Write code that does the following:

1. Define a sample size, and number of repetitions. Also, create an empty array to store the sample means.

2. Write a for loop that loops over the number of repetitions and:
    a. Samples the liberal respondents by the sample size
    b. Calculates its mean
    c. Appends the calculated mean to the array that stores sample means

Using this code, experiment with various sample sizes and number of repetitions. Plot each result. For instance, try the following:

1. Sample size = 20, repetititon = 10
2. Sample size = 100, repetitions = 10
3. Sample size = 100, repetitions = 100000
4. Sample size = 500, repetitions = 100000
5. Sample size = 1000, repetitions = 150000

### Question 5: What happens as you increase the sample size and number of repetitions? How does this property justify the use of statistical methods across a range of problems?